<a href="https://colab.research.google.com/github/xxKeyaxx/Multiple-Floor-Indoor-Positioning-System-using-GraphSage-and-Transfer-Learning/blob/main/Indoor_Positioning_System_Skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-geometric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

test_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_test.csv'
train_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_training.csv'
df_test = pd.read_csv(test_file_path)
df_train = pd.read_csv(train_file_path)
print(df_train.head())  # Preview the data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     100     100     100     100   
3     100     100     100     100     100     100     100     100     100   
4     100     100     100     100     100     100     100     100     100   

   WAP010  ...  WAP587  WAP588  WAP589    Pos_x    Pos_y  Floor_ID  \
0     100  ...     100     100     100  40.7871  31.0272         8   
1     100  ...     100     100     100  40.7871  31.0272         8   
2     100  ...     100     100     100  40.7871  31.0272         8   
3     100  ...     100     100     100  39.4464  30.7985         8   
4     100  ...     100     100     100  39.4464  30.7985  

In [ ]:
# Assuming data is in a Pandas DataFrame
floor_datasets = {}
for floor_id in df_train["Floor_ID"].unique():
    floor_datasets[floor_id] = df_train[df_train["Floor_ID"] == floor_id]

In [ ]:
def preprocess_data(data):
    # Replace 100 with 0 (no detection)
    sensor_cols = [col for col in data.columns if col.startswith('WAP')]
    # data[sensor_cols] = data[sensor_cols].replace(100, 0)
    return data, sensor_cols

df_train, sensor_cols = preprocess_data(df_train)
# df_train = df_train[df_train['Floor_ID'] == 12]

In [ ]:
# # Create graph data for each floor
# def create_graphs_by_floor(data, sensor_cols):
#     graphs = {}
#     for floor_id in data['Floor_ID'].unique():
#         floor_data = data[data['Floor_ID'] == floor_id]
#         x = torch.tensor(floor_data[sensor_cols].values, dtype=torch.float)
#         y = torch.tensor(floor_data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

#         # Create dummy edges (fully connected graph for simplicity)
#         edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
#         edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

#         graphs[floor_id] = Data(x=x, edge_index=edge_index, y=y)
#     return graphs

# graphs_by_floor = create_graphs_by_floor(df_train, sensor_cols)

from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def create_knn_graphs_by_floor(data, sensor_cols, k=5):
    graphs = {}
    for floor_id in data['Floor_ID'].unique():
        floor_data = data[data['Floor_ID'] == floor_id]
        x = torch.tensor(floor_data[sensor_cols].values, dtype=torch.float)
        y = torch.tensor(floor_data[['Pos_x', 'Pos_y', 'Floor_ID']].values, dtype=torch.float)

        # Compute pairwise distances
        distances = euclidean_distances(floor_data[sensor_cols])
        edge_index = []

        # For each node, find k nearest neighbors
        for i in range(distances.shape[0]):
            neighbors = np.argsort(distances[i])[:k]  # Get indices of k nearest neighbors
            for j in neighbors:
                edge_index.append([i, j])

        # Convert edge list to PyTorch Geometric format
        edge_index = torch.tensor(edge_index, dtype=torch.long).T

        graphs[floor_id] = Data(x=x, edge_index=edge_index, y=y)
    return graphs

graphs_by_floor = create_knn_graphs_by_floor(df_train, sensor_cols, k=5)


In [ ]:
# class GraphSAGE(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GraphSAGE, self).__init__()
#         self.conv1 = SAGEConv(in_channels, hidden_channels)
#         self.conv2 = SAGEConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index).relu()
#         x = self.conv2(x, edge_index)
#         return x

from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, concat=True)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Initialize the GAT model
model = GAT(
    in_channels=len(sensor_cols),
    hidden_channels=64,
    out_channels=3,
    heads=4  # Multi-head attention for the first layer
).to(device)


In [ ]:
# # Initialize and train the model for one floor
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GraphSAGE(in_channels=len(sensor_cols), hidden_channels=64, out_channels=2).to(device)

# # Train on a single floor and fine-tune on another
# def train_model(graph_data, model, epochs=100, lr=0.01):
#     graph_data = graph_data.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     criterion = torch.nn.MSELoss()

#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         pred = model(graph_data.x, graph_data.edge_index)
#         loss = criterion(pred, graph_data.y)
#         loss.backward()
#         optimizer.step()
#         if (epoch + 1) % 20 == 0:
#             print(f'Epoch {epoch+1}, Loss: {loss.item()}')
#     return model

def train_model(graph_data, model, epochs=100, lr=0.01):
    graph_data = graph_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(graph_data.x, graph_data.edge_index)
        loss = criterion(pred, graph_data.y)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return model

In [ ]:
max_floor = 0
for floor_id in sorted(graphs_by_floor.keys()):
    print(f"Training/Fine-tuning on Floor {floor_id}...")
    model = train_model(graphs_by_floor[floor_id], model)
    max_floor = floor_id

Training/Fine-tuning on Floor -3...
Epoch 20, Loss: 47339.7421875
Epoch 40, Loss: 4236.4853515625
Epoch 60, Loss: 412.4268798828125
Epoch 80, Loss: 45.94626998901367
Epoch 100, Loss: 16.95070457458496
Training/Fine-tuning on Floor -2...
Epoch 20, Loss: 2941.169921875
Epoch 40, Loss: 47.51955032348633
Epoch 60, Loss: 38.73884582519531
Epoch 80, Loss: 143.7779083251953
Epoch 100, Loss: 47.48019790649414
Training/Fine-tuning on Floor -1...
Epoch 20, Loss: 5273.94873046875
Epoch 40, Loss: 165.50991821289062
Epoch 60, Loss: 90.86602783203125
Epoch 80, Loss: 40.015689849853516
Epoch 100, Loss: 23.661941528320312
Training/Fine-tuning on Floor 0...
Epoch 20, Loss: 1364.7259521484375
Epoch 40, Loss: 118.09388732910156
Epoch 60, Loss: 44.615447998046875
Epoch 80, Loss: 31.02641487121582
Epoch 100, Loss: 22.574495315551758
Training/Fine-tuning on Floor 1...
Epoch 20, Loss: 496.96234130859375
Epoch 40, Loss: 74.24324798583984
Epoch 60, Loss: 70.6664047241211
Epoch 80, Loss: 27.366846084594727
Epoc

In [ ]:
# from sklearn.metrics import r2_score

# # Evaluate the global model on a new dataset
# def evaluate_model(model, test_graph):
#     model.eval()
#     test_graph = test_graph.to(device)
#     with torch.no_grad():
#         pred = model(test_graph.x, test_graph.edge_index)
#         true = test_graph.y

#         # Calculate Mean Absolute Error (MAE)
#         mae = torch.mean(torch.abs(pred - true))

#         # Calculate Root Mean Square Error (RMSE)
#         rmse = torch.sqrt(torch.mean((pred - true) ** 2))

#         r2 = r2_score(true.cpu().numpy(), pred.cpu().numpy())

#     print("Evaluation Results:")
#     print(f"MAE: {mae.item():.4f}")
#     print(f"RMSE: {rmse.item():.4f}")
#     print(f"R2 Score: {r2:.4f}")
#     return mae.item(), rmse.item()


from sklearn.metrics import r2_score

# Evaluate the global model on a new dataset
def evaluate_model(model, test_graph):
    model.eval()
    test_graph = test_graph.to(device)
    with torch.no_grad():
        pred = model(test_graph.x, test_graph.edge_index)  # Predict positions
        true = test_graph.y  # True positions

        # Calculate Mean Absolute Error (MAE)
        mae = torch.mean(torch.abs(pred - true))

        # Calculate Root Mean Square Error (RMSE)
        rmse = torch.sqrt(torch.mean((pred - true) ** 2))

        # Calculate R2 Score
        r2 = r2_score(true.cpu().numpy(), pred.cpu().numpy())

    print("Evaluation Results:")
    print(f"MAE: {mae.item():.4f}")
    print(f"RMSE: {rmse.item():.4f}")
    print(f"R2 Score: {r2:.4f}")
    return mae.item(), rmse.item(), r2


# # Create a unified graph without floor consideration
# def create_unified_graph(data, sensor_cols):
#     # Features: RSSI values
#     x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
#     # Labels: Coordinates (Pos_x, Pos_y)
#     y = torch.tensor(data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

#     # Create a fully connected graph for simplicity
#     num_nodes = x.size(0)
#     edge_index = torch.combinations(torch.arange(num_nodes), r=2).T
#     edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)  # Make edges bidirectional

#     return Data(x=x, edge_index=edge_index, y=y)

def create_knn_unified_graph(data, sensor_cols, k=5):
    # Features: RSSI values
    x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
    # Labels: Coordinates (Pos_x, Pos_y)
    y = torch.tensor(data[['Pos_x', 'Pos_y', 'Floor_ID']].values, dtype=torch.float)

    # Compute pairwise distances
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data[sensor_cols])
    edge_index = []

    # Construct edges using KNN
    for i in range(distances.shape[0]):
        neighbors = np.argsort(distances[i])[:k]  # Get k nearest neighbors
        for j in neighbors:
            edge_index.append([i, j])

    # Convert edge list to PyTorch Geometric format
    edge_index = torch.tensor(edge_index, dtype=torch.long).T

    return Data(x=x, edge_index=edge_index, y=y)


# Load and preprocess the new dataset
df_test, sensor_cols = preprocess_data(df_test)
df_test = df_test[df_test['Floor_ID'] == max_floor]

# Create graph for new dataset
# new_global_graph = create_unified_graph(df_test, sensor_cols)
new_global_graph = create_knn_unified_graph(df_test, sensor_cols, k=5)

# Evaluate the global model on the new graph
print("Evaluating on the new dataset...")
evaluate_model(model, new_global_graph)

Evaluating on the new dataset...
Evaluation Results:
MAE: 4.0871
RMSE: 4.9883
R2 Score: -0.1690


(4.087050914764404, 4.988260746002197, -0.16898374259471893)

In [ ]:
# from torch_geometric.data import DataLoader

# # Loop through each floor's dataset
# models_per_floor = {}
# for floor_id, floor_data in floor_datasets.items():
#     # Preprocess data for the floor (convert to graph)
#     graph_data = preprocess_graph(floor_data)  # Define preprocessing logic

#     # Train a GraphSAGE model
#     model = GraphSAGEModel(input_dim=graph_data.num_node_features, hidden_dim=64, output_dim=2)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         coords_pred, floor_pred = model(graph_data)
#         loss = nn.MSELoss()(coords_pred, graph_data.y[:, :2])  # Only use (Pos_x, Pos_y)
#         loss.backward()
#         optimizer.step()

#     # Save trained model for the floor
#     models_per_floor[floor_id] = model